<a href="https://colab.research.google.com/github/simon108018/Machine-Learning/blob/master/InceptionV3(converter_to_int).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, initializers
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import tensorflow_datasets as tfds
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)
assert float(tf.__version__[:3]) >= 2.3

#tfds.list_builders()

In [ ]:
# 讀取model
model_dir = '/content/drive/My Drive/Colab Notebooks/cifar10/models'
model = tf.keras.models.load_model(model_dir + "/Best-model-InceptionV3-L2.h5")

In [ ]:
# 資料處理
test_data, info = tfds.load("cifar10", split="test", with_info=True)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteODZ0KP/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incompleteODZ0KP/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


In [ ]:
# 定義圖像縮放
def parse_aug_fn(dataset):
    def zoom(x, sh = 299, sw = 299):
        x = tf.image.resize(x, (sh, sw))
        return x
    
    # 影像標準化
    x = tf.cast(dataset['image'], tf.float32) / 255.
    # 影像放大到224*224
    x = zoom(x)
    y = tf.one_hot(dataset['label'], 10)
    y = y
    
    return x, y

In [ ]:
# 資料處理
batch_size = 128
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_data = test_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)
test_data_batch = test_data.batch(batch_size)

In [ ]:
# 評估精準度(用batch加速計算)
model.evaluate(test_data_batch)

79/79 [==============================] - 42s 528ms/step - loss: 0.5511 - categorical_accuracy: 0.8439


[0.5510793924331665, 0.8439000248908997]

In [ ]:
# 轉換成tf.lite形式，並quantize。
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# 以下放入的data只是為了提供Converter測量轉換的範圍
def representative_dataset_gen():
  for data in test_data.batch(1).take(100):
    yield [data[0]]
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_quant_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/tmphfhg8ss6/assets


INFO:tensorflow:Assets written to: /tmp/tmphfhg8ss6/assets


INFO:tensorflow:Assets written to: /tmp/tmp5m130mkh/assets


INFO:tensorflow:Assets written to: /tmp/tmp5m130mkh/assets


INFO:tensorflow:Assets written to: /tmp/tmpyz5rlbk7/assets


INFO:tensorflow:Assets written to: /tmp/tmpyz5rlbk7/assets


In [ ]:
# 儲存tflite model
import pathlib

tflite_models_dir = pathlib.Path("/content/drive/My Drive/Colab Notebooks/cifar10/models")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"cifar10_model.tflite"
tflite_model_file.write_bytes(tflite_float_model)
# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"cifar10_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

24569680